### Data explanatory analysis
I will focus in exploring factors which affect  OR%   and CTR%
I will try to find the best emails subject lines in terms of OR%  and CTR%

In [ ]:
# a code to open excel files as a dataframe 
!pip install openpyxl

In [ ]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import re
%matplotlib inline


In [ ]:
# reading excel file 
df=pd.read_excel("Promotional_emails_subject lines.xlsx")

In [ ]:
df.head(2)

In [ ]:
df.info()

In [ ]:
 # invistigating the subject line column 
df['Subject'].value_counts().reset_index()

I noticed that there is a repetition  for two subject line , but with minor typing mistake , so i merege the similar subject line in one group , because its the same subject line 


In [ ]:
#merging the similar two subject lines 
df['Subject'] = df['Subject'].str.replace('Otroligt ✨3 XXL-format för 249 kr styck', 'Otroligt ✨ 3 XXL-format för 249 kr styck', regex=False)

In [ ]:
#mergig the two subject lines 
df['Subject'] = df['Subject'].str.replace('Använd din kod HA0PJLXC &#x1f525; | Spara nu!', 'Använd din kod 39GX32V0 &#x1f525; | Spara nu!', regex=False)

In [ ]:
df['Subject'] = df['Subject'].str.replace("Upp till 89% rabatt på de mest älskade Canvastavlor! &#x1f929;"
,"Upp till 58% rabatt på de mest älskade Canvastavlor! &#x1F929;"
, regex=False)

### Now, I will get a list of the unique subject lines , later I will engineer features 


In [ ]:
df['Subject'].value_counts().reset_index()

Now I will find the  relation between the email subjects lines  and OR% ( Emails Open Rate )

In [ ]:
df.groupby("Subject")['OR clean %'].mean().reset_index().sort_values(by="OR clean %", ascending=False)

In [ ]:

# I will make a chart for the best five subject lines in terms of OR%

top5 = df.groupby("Subject")['OR clean %'].mean().reset_index().sort_values(by="OR clean %", ascending=False).head(5)

plt.figure(figsize=(10,6))
sns.barplot(x='OR clean %', y='Subject', data=top5, palette='viridis')
plt.xlabel('Average OR clean %')
plt.ylabel('Subject')
plt.title('Top 5 Subjects by OR clean %')
plt.show()


Now I try to Summerize the relation between  emails subject lines and CTR%( Click through Rate )

In [ ]:
df.groupby("Subject")["CTR clean %"].mean().reset_index().sort_values(by="CTR clean %", ascending=False)

In the previous table , the first two rows have unlogigal values  , CTR cannot be more than 1 in this contex , I think its a mistake in data collection , so I will delete the first two rows . beside it is not considered as promotional emails 

In [ ]:
# deleting two rows from the datafram 
df = df[~df['Subject'].isin(['Vill du fortfarande höra från oss?', 'Vi kommer snart att radera dig från mailinglistan!'])]

Now I will check if there is any outliers values in ( OR% , CTR% )

In [ ]:
# afunction to check outliers 

def detect_outliers_in_column_table(df, col):
    if col not in df.columns:
        raise ValueError(f"Column '{col}' not found in DataFrame.")
    
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Filter out the outliers
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)].copy()
    
    # Optional: Add a column showing the outlier value clearly
    outliers['outlier_value'] = outliers[col]
    
    return outliers



In [ ]:

outliers_OR = detect_outliers_in_column_table(df, "OR clean %")
display(outliers_OR)  


In [ ]:
outliers_CTR = detect_outliers_in_column_table(df, "CTR clean %")
display(outliers_CTR)  

There is few outliers values in both OR% and CTR% , this should be revised and make sure it is not a mistake in data collection , I will leave it here because I am doing explanatory data analysis , and I will do any model now !

next , Creating a chart for the relation between email subject lines  and the CTR% 

In [ ]:
df.groupby("Subject")["CTR clean %"].mean().reset_index().sort_values(by="CTR clean %", ascending=False)

In [ ]:
#  a chart for the top best subject lines interms of CTR percentage 
sorted_ctr = df.groupby("Subject")["CTR clean %"].mean().reset_index().sort_values(by="CTR clean %", ascending=False)
subset_ctr = sorted_ctr.iloc[0:10]

plt.figure(figsize=(10,6))
sns.barplot(x='CTR clean %', y='Subject', data=subset_ctr, palette='viridis')
plt.xlabel('Average CTR clean %')
plt.ylabel('Subject')
plt.title('Subjects Ranked top ten by CTR clean %')
plt.show()

Now I will investigate the relation between ( OR% , CTR% ) , and Time ( Quarters , Months , Days of week ) to see if there is any Time series patterns

In [ ]:
#converting date column to a date datatype 
df['Date'] = pd.to_datetime(df['Sent date'])

In [ ]:
# create a month colume
df['Month'] = df['Date'].dt.month


In [ ]:
# OR% for each day of the week
df.groupby("DD")['OR clean %'].mean().reset_index().sort_values(by='OR clean %', ascending=False)

I can not see any relation between day of the weak and OR% 
- This analysis needs more historical data  
- Confirming  that emails are sent the same hour of the day
- Confirming that all subject lines are sent in diffent  weak days 
If after all conditions , still there is no relation , so there might be no relation !!

In [ ]:
df.groupby("DD")["CTR clean %"].mean().reset_index().sort_values(by="CTR clean %", ascending=False)

Saturday is the best day to send emails in terms of CTR%
Monday is the worst day to send emails in terms of CTR%
But there is two conditions must be confirmed 
- first , all emails sent in the same hour of the day 
- Seconf , all subject lines sent in all day weaks 

### Now investegating the monthly performance of OR% and CTR%

In [ ]:
# OR% performance per month
df.groupby("Month")['OR clean %'].mean().reset_index().sort_values(by='OR clean %', ascending=False)

In [ ]:


# Group, average, and sort by Month in ascending order
trend_df = df.groupby("Month")['OR clean %'].mean().reset_index().sort_values(by='Month')

# Plot the trend
plt.figure(figsize=(10,5))
plt.plot(trend_df['Month'], trend_df['OR clean %'], marker='o')
plt.xlabel('Month')
plt.ylabel('Average OR clean %')
plt.title('Time Trend of OR clean % by Month')
plt.grid(True)
plt.show()


There a clear trend shows a drawback in terms of OR%

In [ ]:
#CTR performance per month
df.groupby("Month")["CTR clean %"].mean().reset_index().sort_values(by="CTR clean %", ascending=False)

In [ ]:
# Group, average, and sort by Month in ascending order
trend_df = df.groupby("Month")["CTR clean %"].mean().reset_index().sort_values(by='Month')

# Plot the trend
plt.figure(figsize=(10,5))
plt.plot(trend_df['Month'], trend_df["CTR clean %"], marker='o')
plt.xlabel('Month')
plt.ylabel('Average OR clean %')
plt.title('Time Trend of OR clean % by Month')
plt.grid(True)
plt.show()

The previous chart shows improvment in terms of CTR  

--------------------------------------------------------------------------------------------------------

Understanding the correlation between emails numbers and OR% , and CTR%

In [ ]:
df['Sendings'].corr(df['OR clean %'])

In [ ]:
df['Sendings'].corr(df['CTR clean %'])

There is a correlation between (OR% / CTR%) and the volume of sending emails 

----------------------------------------------------------------------------------------

I prepared a sheet which explains  the features for each subject , in terms of ( number of words , tone , having emoji or not , included price or discount , personalization ) 
I will merge this (subject features sheet) to the current dataframe , so I will have one datafram containning all features 

In [ ]:
subjects_features=pd.read_excel("subject_features1_data.xlsx")

In [ ]:
subjects_features.info()

In [ ]:
# construction a dataframe of emials subjects features and (OR / CTR ) precentages 
merged_df = df.merge(subjects_features, on='Subject', how='left')  


In [ ]:
merged_df.info()

In [ ]:
merged_df.head(2)

#### Now I will study the effect of engeneered features on (OR% and CTR%)

##### 1-Personalization effect 


In [ ]:

merged_df.groupby("Personalization").agg({
    'Sendings': 'sum',
    'OR clean %': 'mean'
}).reset_index().sort_values(by='Sendings', ascending=False)

In [ ]:
merged_df.groupby("Personalization").agg({
    'Sendings': 'sum',
    'CTR clean %': 'mean'
}).reset_index().sort_values(by='Sendings', ascending=False)

Although , there is a clear effect of personalization for OR% , but the volume of emails sent is not balanced in the two categories 

#### 2-Price or Discount effect

In [ ]:
merged_df.groupby("Price or Discount").agg({
    'Sendings': 'sum',
    'OR clean %': 'mean'
}).reset_index().sort_values(by='Sendings', ascending=False)

In [ ]:
merged_df.groupby("Price or Discount").agg({
    'Sendings': 'sum',
    'CTR clean %': 'mean'
}).reset_index().sort_values(by='Sendings', ascending=False)

#### 3- Mentioning of Product effect

In [ ]:
merged_df.groupby("product").agg({
    'Sendings': 'sum',
    'OR clean %': 'mean'
}).reset_index().sort_values(by='Sendings', ascending=False)

In [ ]:
merged_df.groupby("product").agg({
    'Sendings': 'sum',
    'CTR clean %': 'mean'
}).reset_index().sort_values(by='Sendings', ascending=False)

Finally , I will create a data file  to use to build a simple model to predict the OR rate and the CTR 
the features I will use to make predictions are ( the subject of email , month , day of week , number of emails sent )

In [ ]:
merged_df.to_csv("model_data.csv", index=False)